## Vdelave

V prejšnjem primeru smo delali z visoko-dimenzionalnimi vektorji vreče besed dolžine `vocab_size` in izrecno pretvorili nizko-dimenzionalne vektorske predstavitve položajev v redko enotočno predstavitev. Ta enotočna predstavitev ni pomnilniško učinkovita. Poleg tega se vsaka beseda obravnava neodvisno od drugih, zato enotočno kodirani vektorji ne izražajo semantičnih podobnosti med besedami.

V tej enoti bomo nadaljevali raziskovanje podatkovne zbirke **News AG**. Za začetek naložimo podatke in pridobimo nekaj definicij iz prejšnje enote.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Kaj je vdelava?

Ideja **vdelave** (embedding) je predstaviti besede z nižjedimenzionalnimi gostimi vektorji, ki odražajo semantični pomen besede. Kasneje bomo razpravljali o tem, kako zgraditi smiselne vdelave besed, za zdaj pa si vdelave predstavljajmo kot način za zmanjšanje dimenzionalnosti vektorja besede.

Torej, vdelavni sloj prejme besedo kot vhod in ustvari izhodni vektor določene velikosti `embedding_size`. Na nek način je zelo podoben sloju `Dense`, vendar namesto da bi kot vhod prejel vektor z eno vročo kodiranjem (one-hot encoding), lahko sprejme številko besede.

Z uporabo vdelavnega sloja kot prvega sloja v naši mreži lahko preklopimo iz modela vreče besed (bag-of-words) na model **vreče vdelav** (embedding bag), kjer najprej vsako besedo v našem besedilu pretvorimo v ustrezno vdelavo, nato pa izračunamo neko agregatno funkcijo nad vsemi temi vdelavami, na primer `sum`, `average` ali `max`.

![Slika, ki prikazuje klasifikator z vdelavami za pet zaporednih besed.](../../../../../translated_images/sl/embedding-classifier-example.b77f021a7ee67eee.webp)

Naša nevronska mreža za klasifikacijo je sestavljena iz naslednjih slojev:

* Sloj `TextVectorization`, ki prejme niz kot vhod in ustvari tenzor številk tokenov. Določili bomo neko smiselno velikost besedišča `vocab_size` in ignorirali manj pogosto uporabljene besede. Vhodna oblika bo 1, izhodna oblika pa $n$, saj bomo kot rezultat dobili $n$ tokenov, pri čemer vsak vsebuje številke od 0 do `vocab_size`.
* Sloj `Embedding`, ki prejme $n$ številk in vsako številko zmanjša na gosti vektor določene dolžine (v našem primeru 100). Tako se vhodni tenzor oblike $n$ pretvori v tenzor oblike $n\times 100$.
* Agregacijski sloj, ki izračuna povprečje tega tenzorja vzdolž prve osi, tj. izračuna povprečje vseh $n$ vhodnih tenzorjev, ki ustrezajo različnim besedam. Za implementacijo tega sloja bomo uporabili sloj `Lambda` in vanj posredovali funkcijo za izračun povprečja. Izhod bo imel obliko 100 in bo številska predstavitev celotnega vhodnega zaporedja.
* Končni linearni klasifikator `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


V povzetku, v stolpcu **oblika izhoda**, prva dimenzija tenzorja `None` ustreza velikosti minibatcha, druga pa dolžini zaporedja tokenov. Vsa zaporedja tokenov v minibatchu imajo različne dolžine. O tem, kako se spopasti s tem, bomo razpravljali v naslednjem razdelku.

Zdaj pa trenirajmo mrežo:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Opomba**: gradimo vektorizator na podlagi podmnožice podatkov. To je storjeno za pospešitev procesa, kar lahko privede do situacije, ko vsi tokeni iz našega besedila niso prisotni v besedišču. V tem primeru bodo ti tokeni prezrti, kar lahko povzroči nekoliko nižjo natančnost. Vendar pa v resničnem življenju podmnožica besedila pogosto omogoča dobro oceno besedišča.


### Delo z različnimi dolžinami zaporedij spremenljivk

Poglejmo, kako poteka učenje v mini serijah. V zgornjem primeru ima vhodni tenzor dimenzijo 1, uporabljamo pa mini serije dolžine 128, tako da je dejanska velikost tenzorja $128 \times 1$. Vendar je število tokenov v vsakem stavku različno. Če uporabimo plast `TextVectorization` na en sam vhod, je število vrnjenih tokenov različno, odvisno od tega, kako je besedilo tokenizirano:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Vendar, ko uporabimo vektorizator na več zaporedij, mora ustvariti tenzor pravokotne oblike, zato neuporabljene elemente zapolni z oznako PAD (ki je v našem primeru nič):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Tukaj lahko vidimo vdelave:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Opomba**: Da zmanjšamo količino dopolnjevanja, je v nekaterih primerih smiselno razvrstiti vse zaporedje v podatkovnem naboru po naraščajoči dolžini (ali, natančneje, po številu tokenov). To bo zagotovilo, da vsak minibatch vsebuje zaporedja podobne dolžine.


## Semantične vektorske predstavitve: Word2Vec

V našem prejšnjem primeru se je vektorska plast naučila preslikati besede v vektorske predstavitve, vendar te predstavitve niso imele semantičnega pomena. Bilo bi koristno, če bi se naučili vektorske predstavitve, kjer bi bile podobne besede ali sinonimi predstavljeni z vektorji, ki so si blizu glede na neko vektorsko razdaljo (na primer evklidsko razdaljo).

Da to dosežemo, moramo naš model za vektorske predstavitve predhodno naučiti na veliki zbirki besedil z uporabo tehnike, kot je [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ta tehnika temelji na dveh glavnih arhitekturah, ki se uporabljata za ustvarjanje porazdeljenih predstavitev besed:

 - **Neprekinjena vreča besed** (CBoW), kjer model treniramo, da napove besedo na podlagi okoliškega konteksta. Glede na n-gram $(W_{-2},W_{-1},W_0,W_1,W_2)$ je cilj modela napovedati $W_0$ na podlagi $(W_{-2},W_{-1},W_1,W_2)$.
 - **Neprekinjeni preskok-gram** (Continuous skip-gram) je nasproten CBoW. Model uporablja okoliško okno kontekstnih besed za napoved trenutne besede.

CBoW je hitrejši, medtem ko je skip-gram počasnejši, vendar bolje predstavlja redke besede.

![Slika, ki prikazuje algoritma CBoW in Skip-Gram za pretvorbo besed v vektorje.](../../../../../translated_images/sl/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Za eksperimentiranje z Word2Vec vektorskimi predstavitvami, predhodno naučenimi na zbirki podatkov Google News, lahko uporabimo knjižnico **gensim**. Spodaj poiščemo besede, ki so najbolj podobne 'neural'.

> **Opomba:** Ko prvič ustvarjate vektorske predstavitve besed, lahko prenos podatkov traja nekaj časa!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Prav tako lahko iz besede pridobimo vektorsko vdelavo, ki jo uporabimo pri usposabljanju modela za klasifikacijo. Vdelava ima 300 komponent, vendar tukaj za jasnost prikažemo le prvih 20 komponent vektorja:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Odlična stvar pri semantičnih vdelavah je, da lahko manipulirate z vektorskim kodiranjem na podlagi semantike. Na primer, lahko zahtevamo, da najdemo besedo, katere vektorska predstavitev je čim bližje besedama *kralj* in *ženska*, ter čim bolj oddaljena od besede *moški*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Zgornji primer uporablja nekaj notranje magije GenSym, vendar je osnovna logika pravzaprav precej preprosta. Zanimiva stvar pri vdelavah je, da lahko na vektorskih vdelavah izvajate običajne vektorske operacije, kar odraža operacije na pomenskih ravneh besed. Zgornji primer lahko izrazimo z vektorskimi operacijami: izračunamo vektor, ki ustreza **KRALJ-MOŠKI+ŽENSKA** (operaciji `+` in `-` se izvajata na vektorskih predstavitvah ustreznih besed), nato pa poiščemo najbližjo besedo v slovarju temu vektorju:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **OPOMBA**: Dodali smo majhne koeficiente k vektorjema *man* in *woman* – poskusite jih odstraniti, da vidite, kaj se zgodi.

Za iskanje najbližjega vektorja uporabimo TensorFlow orodja za izračun vektorja razdalj med našim vektorjem in vsemi vektorji v besedišču, nato pa z `argmin` poiščemo indeks najmanjše besede.


Medtem ko se Word2Vec zdi odličen način za izražanje semantike besed, ima številne pomanjkljivosti, med drugim naslednje:

* Tako modeli CBoW kot skip-gram so **prediktivni vektorji**, ki upoštevajo le lokalni kontekst. Word2Vec ne izkorišča globalnega konteksta.
* Word2Vec ne upošteva **morfologije** besed, tj. dejstva, da lahko pomen besede temelji na različnih delih besede, kot je koren.

**FastText** poskuša premagati drugo omejitev in gradi na Word2Vec tako, da se uči vektorske predstavitve za vsako besedo in za n-grame znakov, ki jih najde znotraj vsake besede. Vrednosti teh predstavitev se nato povprečijo v en vektor pri vsakem koraku učenja. Čeprav to doda veliko dodatnega računanja pri predtreningu, omogoča vektorskim predstavitvam besed, da kodirajo informacije o podbesedah.

Druga metoda, **GloVe**, uporablja drugačen pristop k vektorskim predstavitvam besed, ki temelji na faktorizaciji matrike beseda-kontekst. Najprej ustvari veliko matriko, ki šteje število pojavitev besed v različnih kontekstih, nato pa poskuša to matriko predstaviti v nižjih dimenzijah na način, ki minimizira izgubo rekonstrukcije.

Knjižnica gensim podpira te vektorske predstavitve besed, in z njimi lahko eksperimentirate tako, da spremenite zgornjo kodo za nalaganje modela.


## Uporaba vnaprej naučenih vektorskih predstavitev v Kerasu

Primer zgoraj lahko prilagodimo tako, da matriko v naši vektorski plasti vnaprej napolnimo s semantičnimi vektorskimi predstavitvami, kot je Word2Vec. Besedišči vnaprej naučenih vektorskih predstavitev in besedilnega korpusa se verjetno ne bosta ujemali, zato moramo izbrati eno. Tukaj raziskujemo dve možni možnosti: uporabo besedišča iz tokenizerja in uporabo besedišča iz Word2Vec predstavitev.

### Uporaba besedišča iz tokenizerja

Pri uporabi besedišča iz tokenizerja bodo nekatere besede iz besedišča imele ustrezne Word2Vec predstavitve, nekatere pa bodo manjkale. Glede na to, da je velikost našega besedišča `vocab_size`, dolžina vektorskih predstavitev Word2Vec pa `embed_size`, bo vektorska plast predstavljena z matriko uteži oblike `vocab_size`$\times$`embed_size`. To matriko bomo napolnili tako, da bomo prešli skozi besedišče:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Za besede, ki niso prisotne v Word2Vec besedišču, lahko pustimo vrednosti kot ničle ali pa ustvarimo naključni vektor.

Zdaj lahko definiramo plast za vdelavo s predhodno naučenimi utežmi:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Opomba**: Upoštevajte, da smo nastavili `trainable=False` pri ustvarjanju `Embedding`, kar pomeni, da sloja Embedding ne bomo ponovno trenirali. To lahko povzroči nekoliko nižjo natančnost, vendar pospeši proces učenja.

### Uporaba besedišča za vdelave

Ena težava pri prejšnjem pristopu je, da se besedišča, uporabljena v TextVectorization in Embedding, razlikujejo. Da bi rešili ta problem, lahko uporabimo eno od naslednjih rešitev:
* Ponovno treniramo model Word2Vec na našem besedišču.
* Naložimo naš nabor podatkov z besediščem iz vnaprej naučenega modela Word2Vec. Besedišča, uporabljena za nalaganje nabora podatkov, je mogoče določiti med nalaganjem.

Drugi pristop se zdi enostavnejši, zato ga bomo implementirali. Najprej bomo ustvarili sloj `TextVectorization` z določenim besediščem, vzetim iz vdelav Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Knjižnica gensim za vektorske predstavitve besed vsebuje priročno funkcijo `get_keras_embeddings`, ki bo samodejno ustvarila ustrezno Kerasovo plast za vektorske predstavitve.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Eden od razlogov, zakaj ne dosegamo večje natančnosti, je ta, da nekaterih besed iz našega nabora podatkov ni v predhodno naučenem GloVe besedišču, zato so v bistvu prezrte. Da bi to presegli, lahko naučimo lastne vdelave na podlagi našega nabora podatkov.


## Kontekstualne vektorske predstavitve

Ena ključnih omejitev tradicionalnih vnaprej naučenih vektorskih predstavitev, kot je Word2Vec, je dejstvo, da kljub temu, da lahko zajamejo določen pomen besede, ne morejo razlikovati med različnimi pomeni. To lahko povzroči težave pri nadaljnjih modelih.

Na primer, beseda 'play' ima različne pomene v teh dveh stavkih:
- Šel sem na **igro** v gledališče.
- John želi **igrati** s svojimi prijatelji.

Vnaprej naučene vektorske predstavitve, o katerih smo govorili, predstavljajo oba pomena besede 'play' z isto vektorsko predstavitvijo. Da bi premagali to omejitev, moramo zgraditi vektorske predstavitve na podlagi **jezikovnega modela**, ki je naučen na velikem korpusu besedil in *ve*, kako se besede lahko povezujejo v različnih kontekstih. Razprava o kontekstualnih vektorskih predstavitvah presega okvir tega vodiča, vendar se bomo k njim vrnili, ko bomo govorili o jezikovnih modelih v naslednji enoti.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
